# Profit分析

## 目的

予測での利益と実際の利益の差分をみてみる。

In [1]:
import os
import datetime
import numpy as np
import pandas as pd

In [2]:
current_dir = %pwd
timestamp = os.path.basename(current_dir)

In [3]:
import sys
sys.path.append('../../')
%cd '../../'

/home/tsu-nera/repo/bakuchi


In [4]:
from src.utils.trade_analysis import TradeAnalysis

%reload_ext autoreload
%autoreload 2

File doesn't exist 


In [5]:
ta = TradeAnalysis(timestamp)

In [6]:
ex1_trades = ta.get_coincheck_trades_df()
ex2_trades = ta.get_liquid_trades_df()

In [7]:
len(ex1_trades), len(ex2_trades)

(95, 95)

In [8]:
ex1_ticks = ta.get_coincheck_ticks_df()
ex2_ticks = ta.get_liquid_ticks_df()

In [9]:
ex1_ticks.head(5)

,bid,ask,tick_timestamp
timestamp,,,
2020-03-25 03:37:38,741906.0,742337.0,2020-03-25 03:37:37
2020-03-25 03:37:39,741906.0,742337.0,2020-03-25 03:37:39
2020-03-25 03:37:40,741906.0,742337.0,2020-03-25 03:37:40
2020-03-25 03:37:41,741906.0,742337.0,2020-03-25 03:37:41
2020-03-25 03:37:42,741906.0,742337.0,2020-03-25 03:37:42


In [10]:
trades = []

for ex1_row, ex2_row in zip(ex1_trades.iterrows(), ex2_trades.iterrows()):
    ex1_item = ex1_row[1]
    ex2_item = ex2_row[1]
    
    ex1_side = ex1_item["side"]
    ex1_timestamp = ex1_item["datetime"]
    ex1_amount = ex1_item["amount"]
    ex1_actual_price = ex1_item["price"]
    ex1_actual_rate = ex1_item["rate"]

    ex2_side = ex2_item["side"]
    ex2_timestamp = ex2_item["datetime"]
    ex2_amount = ex2_item["amount"]
    ex2_actual_price = ex2_item["price"]
    ex2_actual_rate = ex2_item["rate"]
    
    try:
        ex1_tick = ex1_ticks.loc[ex1_timestamp]
    except Exception:
        timestamp = ex1_timestamp + datetime.timedelta(seconds=1)
        try:
            ex1_tick = ex1_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex1_tick = ex1_ticks.loc[timestamp2]

    try:
        ex2_tick = ex2_ticks.loc[ex2_timestamp]
    except Exception:
        timestamp = ex2_timestamp + datetime.timedelta(seconds=1)
        try:
            ex2_tick = ex2_ticks.loc[timestamp]
        except Exception:
            timestamp2 = timestamp + datetime.timedelta(seconds=1)
            ex2_tick = ex2_ticks.loc[timestamp2]

    if ex1_side == "sell":
        ex1_tick_rate = ex1_tick.bid
    else:
        ex1_tick_rate = ex1_tick.ask
    if ex2_side == "sell":
        ex2_tick_rate = ex2_tick.bid
    else:
        ex2_tick_rate = ex2_tick.ask
        
    ex1_expected_price = ex1_tick_rate * ex1_amount
    ex2_expected_price = ex2_tick_rate * ex2_amount

    
    if ex1_side == "buy":
        ex1_expected_price = ex1_expected_price * -1
        ex1_actual_price = ex1_actual_price * -1
    if ex2_side == "buy":
        ex2_expected_price = ex2_expected_price * -1
        ex2_actual_price = ex2_actual_price * -1
        
    expected_profit = sum([ex1_expected_price, ex2_expected_price])
    actual_profit = sum([ex1_actual_price, ex2_actual_price])

    trade = {
        "timestamp": ex1_timestamp, 
        "ex1_side": ex1_side,
        "ex1_expected_rate": ex1_tick_rate,
        "ex1_actual_rate": ex1_actual_rate,
        "ex1_expected_price": ex1_expected_price,
        "ex1_actual_price": ex1_actual_price,
        "ex2_side": ex2_side,
        "ex2_expected_rate": ex2_tick_rate,
        "ex2_actual_rate": ex2_actual_rate,
        "ex2_expected_price": ex2_expected_price,
        "ex2_actual_price": ex2_actual_price,
        "expected_profit": expected_profit,
        "actual_profit": actual_profit
    }
    
    trades.append(trade)

In [11]:
trade_df = pd.DataFrame(trades)
trade_df = trade_df.set_index("timestamp")

In [12]:
profit_df = trade_df[["expected_profit", "actual_profit"]]

In [13]:
int(sum(profit_df["expected_profit"])), int(sum(profit_df["actual_profit"]))

(484, 5)

In [14]:
pd.set_option('display.max_rows', None)
profit_df

,expected_profit,actual_profit
timestamp,,
2020-03-25 03:54:00,8.964464,2.498
2020-03-25 03:55:35,0.330000,-2.682
2020-03-25 04:15:15,16.378658,6.777
2020-03-25 04:15:31,-3.126000,-5.898
2020-03-25 04:16:53,12.387072,4.492
2020-03-25 04:17:22,-0.714000,-1.872
2020-03-25 04:34:10,12.736013,6.428
2020-03-25 04:34:18,0.438000,-2.757
2020-03-25 04:54:00,19.875261,8.017


## coincheck buyにおける売却コインの差分分析

In [15]:
ex1_buy_trades = ex1_trades[ex1_trades["side"] == "buy"]

In [16]:
ex1_buy_amount = ex1_buy_trades["amount"]
min(ex1_buy_amount), max(ex1_buy_amount), np.mean(ex1_buy_amount)

(0.00598557, 0.00601075, 0.005995841276595745)

In [17]:
from decimal import Decimal
num = 0.006 - np.mean(ex1_buy_amount)
"{0:.10f}".format(Decimal(num))

'0.0000041587'

この価格差によって生じた資金移動は

In [18]:
num * len(ex1_buy_trades) * np.mean(ex1_ticks["ask"])

146.32909418789976

## rateの誤差比較

In [22]:
ex1_rate_df = trade_df[["ex1_side", "ex1_expected_rate", "ex1_actual_rate"]].copy()
ex1_rate_df["ex1_rate_diff"] = ex1_rate_df["ex1_expected_rate"] - ex1_rate_df["ex1_actual_rate"]

In [24]:
ex1_rate_df

,ex1_side,ex1_expected_rate,ex1_actual_rate,ex1_rate_diff
timestamp,,,,
2020-03-25 03:54:00,buy,742980.0,743634.0,-654.0
2020-03-25 03:55:35,sell,742990.0,742536.0,454.0
2020-03-25 04:15:15,buy,739987.0,741516.0,-1529.0
2020-03-25 04:15:31,sell,741180.0,740827.0,353.0
2020-03-25 04:16:53,buy,738253.0,739245.0,-992.0
2020-03-25 04:17:22,sell,739410.0,739043.0,367.0
2020-03-25 04:34:10,buy,742094.0,743351.0,-1257.0
2020-03-25 04:34:18,sell,742716.0,742504.5,211.5
2020-03-25 04:54:00,buy,746303.0,748261.0,-1958.0


In [31]:
ex1_diff = ex1_rate_df["ex1_rate_diff"]
max(ex1_diff), min(ex1_diff), np.mean(abs(ex1_diff)), np.std(ex1_diff)

(2233.0, -1978.0, 785.7947368421053, 954.5718181815771)

In [28]:
ex2_rate_df = trade_df[["ex2_side", "ex2_expected_rate", "ex2_actual_rate"]].copy()
ex2_rate_df["ex2_rate_diff"] = ex2_rate_df["ex2_expected_rate"] - ex2_rate_df["ex2_actual_rate"]

In [30]:
ex2_diff = ex2_rate_df["ex2_rate_diff"]
max(ex2_diff), min(ex2_diff), np.mean(abs(ex2_diff)), np.std(ex2_diff)

(2070.0, -693.0, 240.04473684210527, 392.5715229045121)